In [1]:
from neo4j import GraphDatabase

# Creating DiamondDAO data for demo
Here I will first show what normally the chainverse interface would do, aka link the DAO and its users to the different types of elements.

In [2]:
database_uri = "bolt://localhost:7687"
username = "neo4j"
password = "diamondDAO"

In [3]:
class DatabaseConnector:
    "This class defines the connector, and helper functions to interact with the chose schema"
    
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()
        
    def run_query(self, query, props=None):
        try:
            with self.driver.session() as session:
                result = list(session.run(query, props=props))
            return result
        except:
            print(query)
            print(props)
            raise

In [4]:
connector = DatabaseConnector(database_uri, username, password)

In [5]:
# Let's create the DAO
def create_dao(connector, data):
    query = """
        MERGE (diamond:DAO {{id: {}, name: "{}", website:"{}", twitter:"{}"}})
        """.format(data["id"], data["name"], data["website"], data["twitter"])
    connector.run_query(query)
    
diamondDAO = {"id":1, "name": "DiamondDAO", "website":"https://www.diamonddao.xyz/", "twitter":"https://twitter.com/dmnddao"}
GitcoinDAO = {"id":2, "name": "Gitcoin", "website":"https://gitcoin.co/", "twitter":"https://twitter.com/gitcoin"}

create_dao(connector, diamondDAO)
create_dao(connector, GitcoinDAO)

In [6]:
# First matching the different services the DAO runs on
def connect_discourse(connector, dao_id, discourse_name, discourse_id):
    query = """
        MATCH (dao:DAO {{id: {}}}), (discourse:DiscourseCategory {{discourseName: "{}", id: {}}})
        MERGE (discourse) -[edge:isPartOf]-> (dao)
        """.format(dao_id, discourse_name, discourse_id)
    connector.run_query(query)

def connect_DAOHaus(connector, dao_id, daohaus_id):
    query = """
        MATCH (dao:DAO {{id: {}}}), (daohaus:DAOHausDao {{id:{}}})
        MERGE (daohaus) -[edge:isPartOf]-> (dao)
    """.format(dao_id, daohaus_id)
    connector.run_query(query)
    
def connect_Snapshot(connector, dao_id, snapshot_id):
    query = """
        MATCH (dao:DAO {{id: {}}}), (snapshot:SnapshotSpace {{id:"{}"}})
        MERGE (snapshot) -[edge:isPartOf]-> (dao)
    """.format(dao_id, snapshot_id)
    connector.run_query(query)

In [7]:
connect_discourse(connector, 1, "DAOHausDiscourse", 106)
connect_DAOHaus(connector, 1, 792)
connect_DAOHaus(connector, 1, 122)
connect_Snapshot(connector, 2, "gitcoindao.eth")

In [8]:
# Then matching the members
def create_user(connector, user):
    query = """
        MERGE (user:User {{id: {}, name:"{}", twitter:"{}"}})
    """.format(user["id"], user["name"], user["twitter"])
    connector.run_query(query)

def connect_discourse_user(connector, user_id, discourse_username):
    query = """
        MATCH (user:User {{id: {}}}), (discourseUser:DiscourseUser {{username: "{}"}})
        MERGE (user) -[edge:isUser]-> (discourseUser)
    """.format(user_id, discourse_username)
    connector.run_query(query)

def connect_wallet(connector, user_id, address):
    query = """
        MATCH (user:User {{id: {}}}), (wallet:Wallet {{address: "{}"}})
        MERGE (user) -[edge:isWallet]-> (wallet)
    """.format(user_id, address)
    connector.run_query(query)

In [9]:
data = {
    "Xqua": {"name": "XquaInTheMoon", "discourseUsername": "Xqua", "address": "0xF0552F71fDdd72ae6d0F59EfE3fCcFFc39aF21FF".lower(), "twitter": "@XquaInTheMoon"},
    "Jovian": {"name": "Jovian", "discourseUsername": "steffbrowne", "address": "0x3a8af31d3cfe775172853528167bb2b5760bf7b9", "twitter": "@Jovian"},
    "Amphiboly": {"name": "Amphiboly", "discourseUsername": "amphiboly.eth", "address": "0xc75446a6adaef73269dbdece73536977b2b639e0", "twitter": "@amphiboly"},
    "Lemp": {"name": "Lemp", "discourseUsername": "lemp.eth", "address": "0x97b9958facec9acb7adb2bb72a70172cb5a0ea7c", "twitter": "@lemp"},
    "Keating": {"name": "Keating", "address": "0xb8b8bcb1c9e15a9a2325c90b0803e6d225598245", "twitter": "@keating"},
    "FeralChain": {"name": "FeralChain", "address": "0x6a55ae1b0bbbc7edbf346360ddc99b9224205f03", "twitter": "@feralchain"}
}

In [10]:
id_i = 1
for user in data:
    create_user(connector, {"id": id_i, "name": data[user]["name"], "twitter": data[user]["twitter"]})
    if "address" in data[user]:
        connect_wallet(connector, id_i, data[user]["address"])
    if "discourseUsername" in data[user]:
        connect_discourse_user(connector, id_i, data[user]["discourseUsername"])
    id_i += 1

# Let's make a few query examples
Here's a few example queries. First I will look for the DAO users, and how many are voting, and the share distribution

In [11]:
#Let's grab the number of members of the DAO
query = """
MATCH (dao:DAO {id:1})<-[isPartOf]-(daohaus:DAOHausDao)<-[isMemberOf]-(members:Wallet)
RETURN count(DISTINCT members)"""
result = connector.run_query(query)
print("The DAO has {} members accross all DAOHaus instances".format(result[0].value()))

The DAO has 30 members accross all DAOHaus instances


In [12]:
query = "MATCH (users:User) RETURN users"
result = connector.run_query(query)

In [13]:
result

[<Record users=<Node id=629130 labels=frozenset({'User'}) properties={'twitter': '@XquaInTheMoon', 'name': 'XquaInTheMoon', 'id': 1}>>,
 <Record users=<Node id=629131 labels=frozenset({'User'}) properties={'twitter': '@Jovian', 'name': 'Jovian', 'id': 2}>>,
 <Record users=<Node id=629132 labels=frozenset({'User'}) properties={'twitter': '@amphiboly', 'name': 'Amphiboly', 'id': 3}>>,
 <Record users=<Node id=629133 labels=frozenset({'User'}) properties={'twitter': '@lemp', 'name': 'Lemp', 'id': 4}>>,
 <Record users=<Node id=629134 labels=frozenset({'User'}) properties={'twitter': '@keating', 'name': 'Keating', 'id': 5}>>,
 <Record users=<Node id=629135 labels=frozenset({'User'}) properties={'twitter': '@feralchain', 'name': 'FeralChain', 'id': 6}>>]

# Let's get the COvoter network on the DimaondDAO

In [46]:
query = """
MATCH (:DAOHausDao {id:792})<-[:isPartOf]-(proposals:DAOHausProposal)
MATCH (p1:Wallet)-[:hasVotedOn]->(proposals)<-[:hasVotedOn]-(p2:Wallet)
RETURN p1 AS source, p2 AS target, 'COVOTES' as type, count(*) AS weight
"""

results = connector.run_query(query)

In [47]:
# Let's look at a record
results[0]

<Record source=<Node id=616729 labels=frozenset({'Wallet'}) properties={'address': '0xc75446a6adaef73269dbdece73536977b2b639e0'}> target=<Node id=502251 labels=frozenset({'Wallet'}) properties={'address': '0xe04885c3f1419c6e8495c33bdcf5f8387cd88846'}> type='COVOTES' weight=11>

In [5]:
import networkx as nx

In [49]:
G = nx.Graph()
for result in results:
    source = result['source']
    target = result['target']
    weight = result['weight']
    G.add_node(source.id, label=source['address'][:6])
    G.add_node(target.id, label=target['address'][:6])
    G.add_edge(source.id, target.id, weight=weight)

In [50]:
nx.write_graphml(G, 'example_voter_graph.graphml')

# Let's get the DiamondDAO Discourse user network

In [55]:
query = """
MATCH (:DAO {id:1})-[:isPartOf]-(:DiscourseCategory)-[:isPartOf]-(topics:DiscourseTopic)
MATCH (u1:DiscourseUser)-[:isAuthorOf|hasLiked]->(:DiscoursePost)-[:isPartOf]->(topics)<-[:isPartOf]-(:DiscoursePost)<-[:isAuthorOf|hasLiked]-(u2:DiscourseUser)
RETURN u1 AS source, u2 AS target, 'COPARTICIPATE' as type, count(*) AS weight
"""

results = connector.run_query(query)

In [56]:
G = nx.Graph()
for result in results:
    source = result['source']
    target = result['target']
    weight = result['weight']
    G.add_node(source.id, label=source['username'])
    G.add_node(target.id, label=target['username'])
    G.add_edge(source.id, target.id, weight=weight)
nx.write_graphml(G, 'example_DiscourseUser_graph.graphml')

# Writing a Graph in memory for algorithms
We first write a graph, the co-vote graph on DAOHaus, then we use the similarity coeficient to get a score of similarity, and then we order them. 
This gives us a Graph of similar voters

In [63]:
query = """
CALL gds.graph.create.cypher(
    'covotes-graph',
    'MATCH (n:Wallet) RETURN id(n) AS id, labels(n) as labels',
    'MATCH (p1:Wallet)-[:hasVotedOn]->(a:DAOHausProposal)<-[:hasVotedOn]-(p2:Wallet)
     RETURN id(p1) AS source, id(p2) AS target, count(a) AS weight'
)
"""

result = connector.run_query(query)

In [65]:
query = """
CALL gds.nodeSimilarity.write.estimate('covotes-graph', {
  writeRelationshipType: 'SIMILAR',
  writeProperty: 'score'
})
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""

result = connector.run_query(query)
result

[<Record nodeCount=627349 relationshipCount=46822 bytesMin=190792712 bytesMax=205849088 requiredMemory='[181 MiB ... 196 MiB]'>]

In [6]:
query = """
CALL gds.nodeSimilarity.stream('covotes-graph')
YIELD node1, node2, similarity
RETURN gds.util.asNode(node1).address AS Person1, gds.util.asNode(node2).address AS Person2, similarity
ORDER BY similarity DESCENDING, Person1, Person2
"""

results = connector.run_query(query)

In [8]:
G = nx.Graph()
for result in results:
    G.add_edge(result['Person1'], result['Person2'], weight=result['similarity'])

In [12]:
import community as community_louvain

In [13]:
partition = community_louvain.best_partition(G)

In [14]:
for node in G.nodes():
    G.nodes[node]['community'] = partition[node]

In [15]:
nx.write_graphml(G, 'voter_similarity_daohaus.graphml')

In [82]:
G = nx.Graph()
for result in results:
    if result['similarity'] > 0.8:
        G.add_edge(result['Person1'], result['Person2'], weight=result['similarity'])

In [83]:
nx.write_graphml(G, 'voter_similarity_daohaus.thresh-0.8.graphml')